## Estudo sobre a correlação entre ____etc_____etc_____etc_______

# Alunos:
- *Luciano Eduardo*
- *Paulo*
- *Felipe*

<br>
<div class="alert alert-info">
<b>OBJETIVOS: </b>
</div>

O objetivo do trabalho é mostrar de forma gráfica o ... no período de ... para  IMD
Para isso seguiremos os passos a seguir...
1.
2.
3.
  

# Importando as bibliotecas necessárias

In [5]:
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Analisando o grupo http://dados.ufrn.br/group/ensino

In [6]:
# Turmas 2017.2
# http://dados.ufrn.br/dataset/turmas

# turmas
turmas_url = 'http://bit.do/turmas_ufrn_2017_2'

# matrículas em componentes em 2017.2
notas_url = 'http://bit.do/notas_ufrn_2017_2'

# componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

# docentes da ufrn atualizado em 2018
docentes_url = 'http://bit.do/docentes_ufrn_2018'

# turmas 2017.2
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes em 2017.2
notas = pd.read_csv(notas_url, sep=';')

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

docentes = pd.read_csv(docentes_url, sep=';')

# Preparação e limpeza dos dados

In [7]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

In [8]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componenteb
merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57602957,5753061.0,ART2077,LATO SENSU,PRODUÇÃO COREOGRÁFICA I,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T23456
2,57602962,5752714.0,ART2083,LATO SENSU,VIESES ARTÍSTICOS CONTEMPORÂNEOS EM DANÇA ...,DEPARTAMENTO DE ARTES,ABERTA,40.0,1M345 7M2345 17T23456
3,57602963,3031948.0,ART2086,LATO SENSU,PRÁTICAS DE IMPROVISAÇÃO PARA A CENA DAS DANÇA...,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T3456
4,57603192,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
5,57603193,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T456
